In [2]:
import keras
import tensorflow as tf

2024-06-10 20:40:33.946596: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 20:40:34.231145: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 20:40:35.140705: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 20:40:37.502940: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
class MetricEveryN(keras.metrics.Metric):
  """A metric that only runs every `n` batches"""
  def __init__(self, metric_fn, name='custom_metric', n=10, **kwargs):
    super().__init__(name=name, **kwargs)
    self.metric_fn = metric_fn
    self.n = n
    self.total = self.add_weight(name='total', initializer='zeros')
    self.count = self.add_weight(name='count', initializer='zeros')
    self.batch_counter = self.add_weight(name='batch_counter', initializer='zeros')

  @tf.function
  def update_state(self, y_true, y_pred, sample_weight=None):
    self.batch_counter.assign_add(1)

    # Only compute the metric every N batches
    if tf.equal(self.batch_counter % self.n, 0):
      tf.print('Computing')
      result = tf.reduce_mean(self.metric_fn(y_true, y_pred))
      self.total.assign_add(result)
      self.count.assign_add(1)

  def result(self):
    return tf.cond(
      tf.equal(self.count, 0),
      lambda: tf.constant(0.0),
      lambda: self.total / self.count
    )

  def reset_states(self):
    self.total.assign(0.0)
    self.count.assign(0.0)
    self.batch_counter.assign(0.0)


In [9]:
model = keras.Sequential([
  keras.layers.Input(shape=(1,)),
  keras.layers.Dense(1)
])
model.compile(
  loss=keras.losses.MeanSquaredError(),
  optimizer=keras.optimizers.SGD(1e-12),
  metrics=[MetricEveryN(keras.metrics.mean_squared_error, n=10)]
)

X = tf.range(100, dtype=tf.float32)
Y = 2 * X + 1
ds = tf.data.Dataset.from_tensor_slices((X, Y)).batch(4)

model.fit(ds)
# for x, y in ds:
#   z = model(x, training=True)
#   l = model.loss(y, z)
#   print(l)

 1/25 ━━━━━━━━━━━━━━━━━━━━ 9s 383ms/step - custom_metric: 0.0000e+00 - loss: 20.9366Computing
15/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - custom_metric: 2303.8926 - loss: 1756.7682 Computing
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - custom_metric: 6255.1650 - loss: 5027.2100


In [ ]:
for x, y in ds:
  z = model(x, training=True)
  l = model.loss(y, z)
  print(l)
  break

tf.Tensor(nan, shape=(), dtype=float32)
